In [ ]:
from pathlib import Path

import numpy as np

from gettsim import main
from ttsim import InputData, MainTarget, copy_environment

GETTSIM_ROOT = Path.cwd().parent.parent / "src" / "_gettsim"

# Modifying Taxes and Transfers

GETTSIM's design allows you to go beyond the depiction of the current (or historical)
tax and transfer system. Analyzing counterfactual reform scenarios, ranging from small
changes of certain parameters of the tax and transfer system, to the introduction of
large-scale reforms, is a common use case.

This tutorial showcases how to modify the calculation of taxes and transfers when using
GETTSIM.

Here, we focus on small reforms to the means-tested social welfare benefits for
the unemployed (German: Bürgergeld; until 2022: Arbeitslosengeld II). We pick this
example because Bürgergeld is a fairly complex system that uses the entire range of
objects TTSIM offers.

## Status Quo

Before modifying taxes and transfers, it's important to understand how GETTSIM
represents the current tax and transfer system. The core of GETTSIM's implementation is
the **policy environment** - a comprehensive data structure that contains everything
needed to compute taxes and transfers for a specific date.

### What is a Policy Environment?

A policy environment is a nested dictionary that holds all the parameters and functions
needed to calculate taxes and transfers for a given policy date. Think of it as a
complete snapshot of the tax and transfer system at a particular point in time.

### The Three Types of Objects

The policy environment contains three main categories of objects:

1. **Column Objects** (`ColumnObjects`): These work with data columns - either input
   data you provide or results computed by previous functions. They handle the actual
   calculations and data processing.

2. **Parameter Objects** (`ParamObjects`): These store the parameters and constants
   used in calculations, such as tax rates, benefit amounts, or thresholds.

3. **Parameter Functions** (`ParamFunctions`): These process and prepare parameters
   so they can be used by the column objects. They handle parameter transformations
   and validations.

### Getting Started

The first step in modifying taxes and transfers is to create the base policy
environment for the date you want to work with.

In [ ]:
status_quo_environment = main(
    main_target=MainTarget.policy_environment,
    policy_date_str="2025-01-01",
    backend="numpy",
)

We also create some input data in order to verify how our modifications to the policy
environment affect the output. The following input data is required to compute the
amount of social welfare benefits (i.e. `('arbeitslosengeld_2', 'betrag_m_bg')`) when
assuming parental leave benefits (i.e. `('elterngeld', 'betrag_m')`), pensions
(i.e. `('sozialversicherung', 'rente', 'altersrente', 'betrag_m')`,
`('sozialversicherung', 'rente', 'erwerbsminderung', 'betrag_m')`), and unemployment
benefits (i.e. `('sozialversicherung', 'arbeitslosen', 'betrag_m')`) are fixed at some
value.

In [ ]:
INPUT_DATA_TREE = {
    "alter": np.array([40, 40, 5]),
    "alter_monate": np.array([480, 480, 60]),
    "arbeitslosengeld_2": {
        "bezug_im_vorjahr": np.array([False, False, False]),
        "eigenbedarf_gedeckt": np.array([False, False, False]),
        "p_id_einstandspartner": np.array([1, 0, -1]),
    },
    "arbeitsstunden_w": np.array([20, 0, 0]),
    "behinderungsgrad": np.array([0, 0, 0]),
    "einkommensteuer": {
        "abzüge": {
            "beitrag_private_rentenversicherung_m": np.array([0.0, 0.0, 0.0]),
            "kinderbetreuungskosten_m": np.array([0.0, 0.0, 0.0]),
            "p_id_kinderbetreuungskostenträger": np.array([-1, -1, 0]),
        },
        "einkünfte": {
            "aus_forst_und_landwirtschaft": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_gewerbebetrieb": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_kapitalvermögen": {"kapitalerträge_m": np.array([0.0, 0.0, 0.0])},
            "aus_nichtselbstständiger_arbeit": {
                "bruttolohn_m": np.array([1500.0, 0.0, 0.0])
            },
            "aus_selbstständiger_arbeit": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "aus_vermietung_und_verpachtung": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "ist_hauptberuflich_selbstständig": np.array([False, False, False]),
            "sonstige": {
                "alle_weiteren_m": np.array([0.0, 0.0, 0.0]),
                "rente": {
                    "betriebliche_altersvorsorge_m": np.array([0.0, 0.0, 0.0]),
                    "geförderte_private_vorsorge_m": np.array([0.0, 0.0, 0.0]),
                    "sonstige_private_vorsorge_m": np.array([0.0, 0.0, 0.0]),
                },
            },
        },
        "gemeinsam_veranlagt": np.array([True, True, False]),
    },
    "elterngeld": {"betrag_m": np.array([0.0, 0.0, 0.0])},
    "familie": {
        "alleinerziehend": np.array([False, False, False]),
        "p_id_ehepartner": np.array([1, 0, -1]),
        "p_id_elternteil_1": np.array([-1, -1, 0]),
        "p_id_elternteil_2": np.array([-1, -1, 1]),
    },
    "geburtsjahr": np.array([1985, 1985, 2020]),
    "hh_id": np.array([0, 0, 0]),
    "kindergeld": {
        "in_ausbildung": np.array([False, False, False]),
        "p_id_empfänger": np.array([-1, -1, 0]),
    },
    "p_id": np.array([0, 1, 2]),
    "sozialversicherung": {
        "arbeitslosen": {"betrag_m": np.array([0.0, 0.0, 0.0])},
        "kranken": {"beitrag": {"privat_versichert": np.array([False, False, False])}},
        "pflege": {"beitrag": {"hat_kinder": np.array([True, True, False])}},
        "rente": {
            "altersrente": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "bezieht_rente": np.array([False, False, False]),
            "erwerbsminderung": {"betrag_m": np.array([0.0, 0.0, 0.0])},
            "jahr_renteneintritt": np.array([2060, 2060, 2090]),
        },
    },
    "unterhalt": {"tatsächlich_erhaltener_betrag_m": np.array([0.0, 0.0, 0.0])},
    "vermögen": np.array([0.0, 0.0, 0.0]),
    "wohnen": {
        "bewohnt_eigentum_hh": np.array([False, False, False]),
        "bruttokaltmiete_m_hh": np.array([600.0, 600.0, 600.0]),
        "heizkosten_m_hh": np.array([60.0, 60.0, 60.0]),
        "wohnfläche_hh": np.array([50.0, 50.0, 50.0]),
    },
    "wohngeld": {"mietstufe_hh": np.array([4, 4, 4])},
}

The status quo is the following:

In [ ]:
main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"arbeitslosengeld_2": {"betrag_m_bg": None}}},
    include_warn_nodes=False,
)

## Modifying Parameters

GETTSIM's parameters are stored in different objects depending on their type. If you
modify the parameters in the `policy_environment`, you will encounter the following
objects:

1. **ScalarParam**: A scalar parameter, i.e. a parameter that is a single number.
2. **DictParam**: A parameter that is a flat dictionary with homogeneous keys and values
   (i.e. all keys and values are of the same type).
3. **ConsecutiveIntLookupTableParam**: A lookup table that stores values and assigns a
   consecutive integer index to each value.
4. **PiecewisePolynomialParam**: A piecewise polynomial parameter, i.e. a parameter that
    describes a piecewise polynomial function.
5. **RawParam**: A parameter that does not fit into the other categories. For these
   parameters, we need `ParamFunction`s to process them (see next section).

Any of those parameter classes has the following attributes:
- `leaf_name`: The leaf name of the parameter in GETTSIM's policy environment.
- `start_date`: The date from which the parameter is valid (if applicable).
- `end_date`: The date until which the parameter is valid (if applicable).
- `unit`: The unit of the parameter (if applicable).
- `reference_period`: The period over which the parameter is valid (if applicable).
- `name`: The name of the parameter.
- `description`: A more elaborate description of the parameter.
- `value`: The value of the parameter.
- `note`: Some notes (if applicable).
- `reference`: A legal reference.

When modifying parameters, you will mostly care about the `value` attribute.

### Scalar Parameters

Scalar parameters are the simplest type of parameters. They are represented by the
`ScalarParam` class. They are stored as a single number in the `policy_environment`.

Let's take a look at the `kindersofortzuschlag` parameter. This parameter increases the
transfer to children by a fixed amount.

As you can see the `kindersofortzuschlag` parameter is a `ScalarParam` object and its
value is 25€ in the status quo.

In [ ]:
status_quo_environment["arbeitslosengeld_2"]["kindersofortzuschlag"]

In [ ]:
status_quo_environment["arbeitslosengeld_2"]["kindersofortzuschlag"].value

Let's increase the parameter.

#### Step 1: Create a copy of the status quo policy environment. 

This is good practice to avoid inplace modifications of the original policy environment.

In [ ]:
higher_kindersofortzuschlag_policy_environment = copy_environment(
    status_quo_environment
)

#### Step 2: Create the new parameter.

Create a new `ScalarParam` object. To do this, we first import the `ScalarParam` class
from GETTSIM and then instantiate it with the new value.

**Tip**: You don't have to specify all attributes of the `ScalarParam` class. Only the
value attribute is required.

In [ ]:
from ttsim.tt_dag_elements.param_objects import ScalarParam

new_kindersofortzuschlag = ScalarParam(value=40)

#### Step 3: Replace the old parameter with the new one in the new policy environment

In [ ]:
higher_kindersofortzuschlag_policy_environment["arbeitslosengeld_2"][
    "kindersofortzuschlag"
] = new_kindersofortzuschlag

Let's call GETTSIM with the modified policy environment.

In [ ]:
main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"arbeitslosengeld_2": {"betrag_m_bg": None}}},
    policy_environment=higher_kindersofortzuschlag_policy_environment,
    include_warn_nodes=False,
)

### Dict Parameters

Dict parameters are parameters that are a flat dictionary with homogeneous keys and
values. They are represented by the `DictParam` class. They are stored as a flat
dictionary in the `policy_environment`.

Let's take a look at the `berechtigte_wohnfläche_miete` parameter. This parameter
contains the amount of the admissible housing size for each type of housing.

As you can see the `berechtigte_wohnfläche_miete` parameter is a `DictParam` object and
its value is a dictionary.

In [ ]:
status_quo_environment["arbeitslosengeld_2"]["berechtigte_wohnfläche_miete"]

In [ ]:
status_quo_environment["arbeitslosengeld_2"]["berechtigte_wohnfläche_miete"].value

Let's modify the parameter by decreasing the 

In [ ]:
higher_kindersofortzuschlag_policy_environment = copy_environment(
    status_quo_environment
)

#### Step 2: Create the new parameter.

Create a new `ScalarParam` object. To do this, we first import the `ScalarParam` class
from GETTSIM and then instantiate it with the new value.

**Tip**: You don't have to specify all attributes of the `ScalarParam` class. Only the
value attribute is required.

In [ ]:
from ttsim.tt_dag_elements.param_objects import ScalarParam

new_kindersofortzuschlag = ScalarParam(value=40)

#### Step 3: Replace the old parameter with the new one in the new policy environment

In [ ]:
higher_kindersofortzuschlag_policy_environment["arbeitslosengeld_2"][
    "kindersofortzuschlag"
] = new_kindersofortzuschlag

Let's call GETTSIM with the modified policy environment.

In [ ]:
main(
    main_target=MainTarget.results.df_with_nested_columns,
    policy_date_str="2025-01-01",
    input_data=InputData.tree(INPUT_DATA_TREE),
    tt_targets={"tree": {"arbeitslosengeld_2": {"betrag_m_bg": None}}},
    policy_environment=higher_kindersofortzuschlag_policy_environment,
    include_warn_nodes=False,
)

## Modifying Parameter Functions

## Modifying Column Objects